<h4> Configurações do notebook

In [ ]:
%load_ext autoreload
%autoreload 2
%cd ..

<h4> Importando as bibliotecas

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

<h4> Abrindo o dataset

In [6]:
#Carregando o dataset 
df = pd.read_csv("data/Newdata.csv")

In [7]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Card Type,Point Earned,Churn
0,709,Spain,Male,35,2,0.00,2,1,0,104982.39,GOLD,422,0
1,744,France,Male,29,1,43504.42,1,1,1,119327.75,PLATINUM,607,0
2,773,France,Male,64,2,145578.28,1,0,1,186172.85,No card,630,0
3,646,Germany,Female,29,4,105957.44,1,1,0,15470.91,PLATINUM,345,0
4,675,France,Female,57,8,0.00,2,0,1,95463.29,No card,632,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,636,France,Female,39,3,118336.14,1,1,0,184691.77,GOLD,349,0
7996,528,France,Male,35,3,156687.10,1,1,0,199320.77,PLATINUM,982,0
7997,622,France,Male,26,9,0.00,2,1,1,153237.59,PLATINUM,344,0
7998,592,France,Male,28,5,137222.77,1,0,0,39608.58,No card,282,0


<h2> Feature engineering, split e pré-processamento

Para obter a melhor performance possível dos modelos de testes, é necessário realizar algumas transformações e criações de features que impulsionem o rendimento dos algoritmos. Abaixo explico melhor as features e as transformações do pré-processador que vou desenvolver

Pré-processador:

Como apenas modelos baseados em árvores serão utilizados, o pré-processador terá:
- Nenhuma transformação em colunas numéricas, pois modelos de árvores não precisam que os dados estejam na mesma escala para performarem bem.
- Para preservar a característica ordinal da coluna "Card Type", será aplicado Ordinal Encoding
- Para as colunas "Geography" e "Gender", será aplicado One-Hot Encoding. Isso funcionará bem pois há poucos valores únicos nessa coluna (2 em Gender e 3 em Geography), assim o dataset expandirá pouco e não afetará a qualidade do modelo.

Feature engineering:  

Afim de extrair o máximo de informações relevantes possíveis do dataset e melhorar o desempenho do algoritmo, algumas features serão criadas. Todas as features criadas serão determinísticas e não causarão data leakage, mesmo sendo realizadas antes do split dos dados. As features são:
- Engagement (HasCrCard + IsActiveMember + NumOfProducts)
- Financial_Value (Balance + EstimatedSalary)
- Age_per_Tenure (Age / (Tenure + 0.1))
- Balance_per_Tenure (Balance / (Tenure + 1))
- Products_per_Tenure (NumOfProducts / (Tenure + 1))
- Points_per_Tenure (Point Earned / (Tenure + 1))
- FinancialValue_per_Tenure (Financial_Value / (Tenure + 1))
- Engagement_per_Age (Engagement / Age)
- Balance_per_Products (Balance / NumOfProducts)
- Older_Inactive (Cliente com mais de 50 anos e inativo)
- Low_value_Churner (Clientes inativos, com balanço igual a 0 e com 0 ou 1 produtos adquiridos)

<h3> 1. Feature engineering

In [8]:
#Cópia do dataset carregado
df_train = df.copy()

#Criação das features
df_train["Engagement"] = (df_train['HasCrCard'] + df_train['IsActiveMember'] + df_train['NumOfProducts'])
df_train["Financial_value"] = (df_train['Balance'] + df_train['EstimatedSalary'])
df_train['Older_Inactive'] = ((df_train['Age'] >= 50) & (df_train['IsActiveMember'] == 0)).astype(int)
df_train['Low_value_Churner'] = ((df_train['IsActiveMember'] == 0) &(df_train['Balance'] == 0) &(df_train['NumOfProducts'] <= 1)).astype(int)
df_train['Age_per_Tenure'] = (df_train['Age'] / (df_train['Tenure'] + 0.1)).astype('float64')
df_train['Balance_per_Tenure'] = (df_train['Balance'] / (df_train['Tenure'] + 0.1)).astype('float64')
df_train['Products_per_Tenure'] = (df_train['NumOfProducts'] / (df_train['Tenure'] + 1)).astype('float64')
df_train['Points_per_Tenure'] = (df_train['Point Earned'] / (df_train['Tenure'] + 1)).astype('float64')
df_train['FinancialValue_per_Tenure'] = (df_train['Financial_value'] / (df_train['Tenure'] + 1)).astype('float64')
df_train['Engagement_per_Age'] = (df_train['Engagement'] / (df_train['Age'])).astype('float64')
df_train['Balance_per_Products'] = (df_train['Balance'] / (df_train['NumOfProducts'])).astype('float64')

In [9]:
df_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,...,Financial_value,Older_Inactive,Low_value_Churner,Age_per_Tenure,Balance_per_Tenure,Products_per_Tenure,Points_per_Tenure,FinancialValue_per_Tenure,Engagement_per_Age,Balance_per_Products
0,709,Spain,Male,35,2,0.00,2,1,0,104982.39,...,104982.39,0,0,16.666667,0.000000,0.666667,140.666667,34994.130000,0.085714,0.00
1,744,France,Male,29,1,43504.42,1,1,1,119327.75,...,162832.17,0,0,26.363636,39549.472727,0.500000,303.500000,81416.085000,0.103448,43504.42
2,773,France,Male,64,2,145578.28,1,0,1,186172.85,...,331751.13,0,0,30.476190,69322.990476,0.333333,210.000000,110583.710000,0.031250,145578.28
3,646,Germany,Female,29,4,105957.44,1,1,0,15470.91,...,121428.35,0,0,7.073171,25843.278049,0.200000,69.000000,24285.670000,0.068966,105957.44
4,675,France,Female,57,8,0.00,2,0,1,95463.29,...,95463.29,0,0,7.037037,0.000000,0.222222,70.222222,10607.032222,0.052632,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,636,France,Female,39,3,118336.14,1,1,0,184691.77,...,303027.91,0,0,12.580645,38172.948387,0.250000,87.250000,75756.977500,0.051282,118336.14
7996,528,France,Male,35,3,156687.10,1,1,0,199320.77,...,356007.87,0,0,11.290323,50544.225806,0.250000,245.500000,89001.967500,0.057143,156687.10
7997,622,France,Male,26,9,0.00,2,1,1,153237.59,...,153237.59,0,0,2.857143,0.000000,0.200000,34.400000,15323.759000,0.153846,0.00
7998,592,France,Male,28,5,137222.77,1,0,0,39608.58,...,176831.35,0,0,5.490196,26906.425490,0.166667,47.000000,29471.891667,0.035714,137222.77


<h3> 2. Split

Antes de começar efetivamente a construir o pré-processador, farei o split dos dados. Como já dito anteriormente, não há possibilidade de vazamento com as features criadas anteriormente.

In [10]:
X = df_train.drop(columns={"Churn"})
y = df_train["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

O parâmetro stratify=y foi usado para manter a mesma proporção de registros com churn e sem churn em cada dataset

In [ ]:
#Tamanho dos datasets de treino e teste
print(f'O dataset de treino tem {X_train.shape[0]} linhas')
print(f'O dataset de teste tem {X_test.shape[0]} linhas')

O dataset de treino tem 6400 linhas
O dataset de teste tem 1600 linhas


In [12]:
#Proporção de churn em cada dataset
print(f'Proporcão de churn no dataset de treino')
print(f'{y_train.value_counts(normalize=True)}')
print(f'\nProporcão de churn no dataset de teste')
print(f'{y_test.value_counts(normalize=True)}')

Proporcão de churn no dataset de treino
Churn
0    0.79625
1    0.20375
Name: proportion, dtype: float64

Proporcão de churn no dataset de teste
Churn
0    0.79625
1    0.20375
Name: proportion, dtype: float64


<h3> 3. Pré-processamento

Abaixo irei codificar o pré-processador seguindo as técnicas que mencionei

In [13]:
#Colunas do One-Hot e do ordinal encoder
ohe_cols = ['Geography', 'Gender']
ordinal_cols = ['Card Type']

#Hierarquia do ordinal encoder
ordinal_categories = [['No card', 'SILVER', 'GOLD', 'DIAMOND', 'PLATINUM']]

print(f"As colunas utilizadas no One-Hot Encoder são: {ohe_cols}")
print(f"As colunas utilizadas no Ordinal Encoder são: {ordinal_cols}")

As colunas utilizadas no One-Hot Encoder são: ['Geography', 'Gender']
As colunas utilizadas no Ordinal Encoder são: ['Card Type']


In [14]:
#Pré-processador
preprocessor = ColumnTransformer([
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), ohe_cols),
    ('ordinal_encoder', OrdinalEncoder(categories=ordinal_categories), ordinal_cols)],
    remainder='passthrough'
)

In [15]:
#Utilizando o pré-processador no dataset de treino
X_train_processed = preprocessor.fit_transform(X_train, y_train)
X_train_processed.shape

(6400, 26)

<h2> Treino e avaliação dos resultados

Nessa etapa do estudo, irei comparar 3 modelos diferentes baseados em árvores, com o intuito de verificar qual o melhor algoritmo para esse problema. Os modelos são: Decision Tree Classifier, Random Forest Classifier e Light GBM Classifier

Para a comparação, usarei uma técnica chamada validação cruzada K-fold estratificada (Stratified K-fold cross validation) para treinar cada modelo. A K-fold cross validation consiste em dividir um dataset em K subsets, mantendo a proporção de churn igual em cada um. Após isso, são feitas iterações onde K-1 subsets são utilizados para treinamento do modelo e o restante para teste, repetindo-se o processo até que todos os subconjuntos tenham sido usados como conjunto de avaliação

In [16]:
#Algoritmos utilizados
tree_models = {
              'Decision Tree': DecisionTreeClassifier(),
              'Random Forest': RandomForestClassifier(),
              'LightGBM': LGBMClassifier()
              }

In [ ]:
#K-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

for name, model in tree_models.items():
    #Validação cruzada
    scores = cross_val_score(
        model,
        X_train_processed,
        y_train,
        cv=skf,
        scoring='roc_auc',
        n_jobs=-1
    )

    model.fit(X_train_processed, y_train)

    y_train_proba = model.predict_proba(X_train_processed)[:, 1]
    train_score = roc_auc_score(y_train, y_train_proba)

    results.append({
        'Model': name,
        'ROC_AUC_Mean': scores.mean(),
        'Training_Score': train_score
    })

In [18]:
#Transformando os resultados em um dataset
eval_df = pd.DataFrame(results)

eval_df

,Model,ROC_AUC_Mean,Training_Score
0,Decision Tree,0.696146,1.000000
1,Random Forest,0.851368,1.000000
2,LightGBM,0.850072,0.989226


Avaliação:
- Decision Tree: Apresentou forte sobreajuste, com desempenho perfeito no treino, mas baixo na validação. Em geral, isso indica que o modelo decorou os dados de treinamento e com isso não consegue generalizar bem para dados novos, mostrando ser um algoritmo com desempenho inferior.
- Random Forest: Embora tenha também tenha apresentado sobreajuste, assim como a Decision Tree, A Random Forest obteve um bom desempenho com médias de ROC-AUC próximas a 0,85.
- LightGBM: Dentre os três algoritmos, o LightGBM alcançou o melhor resultado. Além de ter o melhor ROC-AUC (acima de 0,85), também foi o modelo que teve a menor diferença nas médias de treino e validação, com apenas 0,15.

<h2> Conclusão

Em desenvolvimento